# Parent-Document Retriever
* 유사문서의 부모문서를 참고한다.

In [25]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [5]:
# loaders = [
#     PyPDFLoader("/content/drive/MyDrive/강의 자료/[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf"),
#     PyPDFLoader("/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"),
# ]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load_and_split())


loader = PyPDFLoader("./assets/E-9(비전문취업 비자) 발급, 변경, 연장.pdf")
docs = loader.load_and_split()

In [6]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=ko_embedding
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [8]:
retriever.add_documents(docs, ids=None)

In [11]:
sub_docs = vectorstore.similarity_search("한국에서의 체류 기간을 어떻게 연장할 수 있나요?")

In [12]:
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 496


체류자격
변경허가
●
기타
(
G-1)
자격
소지자의
비전문취업
(
E-9)
자격
회복절차
○
대상자
■
고용허가제
등으로
입국
후
산업재해
치료
등을
위해
기타
(
G-1)
자격으로
변경
후
국내체류
중이던
외국인근로자
중
국내
체류기간
상한이
도래하지
않은
자
○
체류기간
상한
(
입국일로부터
기산
)
■
비전문취업자
(
E-9)
3
년
,
다만
3
년
만기
체류자
중
고용주가
재고용한
경우
1
년
10
개월
이내
체류기간
체류기간
연장
허가
●
제출
서류
○
신청서
(
별지
34
호
서식
),
여권
및
외국인등록증
,
수수료
○
고용허가서
사본
○
표준근로계약서
사본
○
사업자등록증
사본
○
입국
후
3
년
만료
재고용에
따른
최대
1
년
10
개월
추가
연장의
경우
‘
취업기간만료자
취업활동기간
연장확인서
(
고용노동부
발급
)’
○
체류지
입증서류
(
임대차계약서
,
숙소제공
확인서
,
체류기간
만료예고
통지우편물
,
공공요금
납부영수증
,
기숙사비
영수증
등
)
●


In [13]:
retrieved_docs = retriever.get_relevant_documents("한국에서의 체류 기간을 어떻게 연장할 수 있나요?")

In [14]:
print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 1006


체류자격
변경허가
●
기타
(
G-1)
자격
소지자의
비전문취업
(
E-9)
자격
회복절차
○
대상자
■
고용허가제
등으로
입국
후
산업재해
치료
등을
위해
기타
(
G-1)
자격으로
변경
후
국내체류
중이던
외국인근로자
중
국내
체류기간
상한이
도래하지
않은
자
○
체류기간
상한
(
입국일로부터
기산
)
■
비전문취업자
(
E-9)
3
년
,
다만
3
년
만기
체류자
중
고용주가
재고용한
경우
1
년
10
개월
이내
체류기간
체류기간
연장
허가
●
제출
서류
○
신청서
(
별지
34
호
서식
),
여권
및
외국인등록증
,
수수료
○
고용허가서
사본
○
표준근로계약서
사본
○
사업자등록증
사본
○
입국
후
3
년
만료
재고용에
따른
최대
1
년
10
개월
추가
연장의
경우
‘
취업기간만료자
취업활동기간
연장확인서
(
고용노동부
발급
)’
○
체류지
입증서류
(
임대차계약서
,
숙소제공
확인서
,
체류기간
만료예고
통지우편물
,
공공요금
납부영수증
,
기숙사비
영수증
등
)
●
취업활동기간
연장을
받은
비전문취업자에
대한
체류기간
연장
허가
○
적용대상
■
‘09.
12.
10.
이후에
외국인근로자
고용
등에
관한
법률
시행규칙에
따라
고용주가
재고용을
신청하여
’
취업기간
만료자
취업활동기간
연장확인서
‘
【
붙임
2
】
를
발급받은
비전문취업자
●
구직신청자
특례
○
대상
■
구직등록
유효기간
(
구직신청일로부터
3
개월
)
이전에
체류기간만료일이
도래하는
자로서
사업장변경
가능
횟수가
남아
있는
자
○
허가기간
■
구직등록필증
발급일로부터
90
일
범위
내
○
제출서류
■
신청서
(
별지
34
호서식
),
여권
및
외국인등록증
,
수수료
없음
■
자진출국
각서
■
구직등록필증
■
체류지
입증서류
(
임대차계약서
,
숙소제공
확인서
,
체류기간
만료예고
통지우편물
,
공공요금
납부영수증
,
기숙사비
영수증
등
)
●
민원신청서
상
체류지는
기재하되
체류지
입증서류
제출이
곤란할
경우
생략가능
.
단
,
체류지
정보를
허위기재한
경우
출

## 본문의 Full_chunk가 너무 길때

In [15]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=800)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=ko_embedding
)
# The storage layer for the parent documents
store = InMemoryStore()

In [27]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [17]:
parent_document_retriever.add_documents(docs)

In [18]:
len(list(store.yield_keys()))

22

In [19]:
sub_docs = vectorstore.similarity_search("한국에서의 체류 기간을 어떻게 연장할 수 있나요?")

In [20]:
print(sub_docs[0].page_content)

○
입국
후
3
년
만료
재고용에
따른
최대
1
년
10
개월
추가
연장의
경우
‘
취업기간만료자
취업활동기간
연장확인서
(
고용노동부
발급
)’
○
체류지
입증서류
(
임대차계약서
,
숙소제공
확인서
,
체류기간
만료예고
통지우편물
,
공공요금
납부영수증
,
기숙사비
영수증
등
)
●
취업활동기간
연장을
받은
비전문취업자에
대한
체류기간
연장
허가
○


In [21]:
len(sub_docs[0].page_content)

198

In [22]:
retrieved_docs = parent_document_retriever.get_relevant_documents("한국에서의 체류 기간을 어떻게 연장할 수 있나요?")

In [23]:
print(retrieved_docs[0].page_content)

체류자격
변경허가
●
기타
(
G-1)
자격
소지자의
비전문취업
(
E-9)
자격
회복절차
○
대상자
■
고용허가제
등으로
입국
후
산업재해
치료
등을
위해
기타
(
G-1)
자격으로
변경
후
국내체류
중이던
외국인근로자
중
국내
체류기간
상한이
도래하지
않은
자
○
체류기간
상한
(
입국일로부터
기산
)
■
비전문취업자
(
E-9)
3
년
,
다만
3
년
만기
체류자
중
고용주가
재고용한
경우
1
년
10
개월
이내
체류기간
체류기간
연장
허가
●
제출
서류
○
신청서
(
별지
34
호
서식
),
여권
및
외국인등록증
,
수수료
○
고용허가서
사본
○
표준근로계약서
사본
○
사업자등록증
사본
○
입국
후
3
년
만료
재고용에
따른
최대
1
년
10
개월
추가
연장의
경우
‘
취업기간만료자
취업활동기간
연장확인서
(
고용노동부
발급
)’
○
체류지
입증서류
(
임대차계약서
,
숙소제공
확인서
,
체류기간
만료예고
통지우편물
,
공공요금
납부영수증
,
기숙사비
영수증
등
)
●
취업활동기간
연장을
받은
비전문취업자에
대한
체류기간
연장
허가
○
적용대상
■
‘09.
12.
10.
이후에
외국인근로자
고용
등에
관한
법률
시행규칙에
따라
고용주가
재고용을
신청하여
’
취업기간
만료자
취업활동기간
연장확인서
‘
【
붙임
2
】
를
발급받은
비전문취업자
●
구직신청자
특례
○
대상
■
구직등록
유효기간
(
구직신청일로부터
3
개월
)
이전에
체류기간만료일이
도래하는
자로서
사업장변경
가능
횟수가
남아
있는
자
○
허가기간
■
구직등록필증
발급일로부터
90
일
범위
내
○
제출서류
■
신청서
(
별지
34
호서식
),


In [24]:
len(retrieved_docs[0].page_content)

800

In [28]:
def gpt_call(model_name: str, query: str):
    llm = ChatOpenAI(model_name=model_name, temperature = 0)
    qa = RetrievalQA.from_chain_type(
        llm = llm, 
        chain_type = "stuff", 
        retriever = parent_document_retriever,
        return_source_documents = True
    )
    result = qa(query)

    print(f"Result: {result['result']}\n")
    for i in result['source_documents']:
        print(i.metadata)
        print("-"*100)
        print(i.page_content)
        print("-"*100)
        
    return result

In [29]:
query = "한국에서의 체류 기간을 어떻게 연장할 수 있나요?"
# query = "베트남으로 일시 귀국한 후 E-9 비자로 한국에 재입국하는 과정 및 기준은 무엇인가요?"
# query = "E-9 비자 소지자가 한국에서 취업할 수 있는 구체적인 산업 분야와 직업은 무엇인가요?"

gpt_call(model_name="gpt-4", query=query)

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Result: 한국에서의 체류 기간을 연장하려면 다음과 같은 절차를 따르면 됩니다:

1. 신청서(별지 34호 서식)를 작성합니다.
2. 여권 및 외국인등록증을 준비합니다.
3. 수수료를 지불합니다.
4. 고용허가서 사본을 제출합니다.
5. 표준근로계약서 사본을 제출합니다.
6. 사업자등록증 사본을 제출합니다.
7. 입국 후 3년 만료 재고용에 따른 최대 1년 10개월 추가 연장의 경우 '취업기간만료자 취업활동기간 연장확인서(고용노동부 발급)'를 제출합니다.
8. 체류지 입증서류(임대차계약서, 숙소제공 확인서, 체류기간 만료예고 통지우편물, 공공요금 납부영수증, 기숙사비 영수증 등)를 제출합니다.

이러한 절차를 따르면 체류 기간을 연장할 수 있습니다. 그러나 이는 비전문취업자(E-9)에 대한 정보이며, 다른 체류자격에 따라 요구되는 서류와 절차가 다를 수 있습니다. 따라서 자신의 체류자격에 맞는 정보를 확인하는 것이 중요합니다.

{'source': './assets/E-9(비전문취업 비자) 발급, 변경, 연장.pdf', 'page': 10}
----------------------------------------------------------------------------------------------------
체류자격
변경허가
●
기타
(
G-1)
자격
소지자의
비전문취업
(
E-9)
자격
회복절차
○
대상자
■
고용허가제
등으로
입국
후
산업재해
치료
등을
위해
기타
(
G-1)
자격으로
변경
후
국내체류
중이던
외국인근로자
중
국내
체류기간
상한이
도래하지
않은
자
○
체류기간
상한
(
입국일로부터
기산
)
■
비전문취업자
(
E-9)
3
년
,
다만
3
년
만기
체류자
중
고용주가
재고용한
경우
1
년
10
개월
이내
체류기간
체류기간
연장
허가
●
제출
서류
○
신청서
(
별지
34
호
서식
),
여권
및
외국인등록증
,
수수료
○
고용허가서
사본
○
표준근로계약서
사본
○
사업자등록증
사본
○
입국
후
3
년
만료
재고용에
따른
최대
1


{'query': '한국에서의 체류 기간을 어떻게 연장할 수 있나요?',
 'result': "한국에서의 체류 기간을 연장하려면 다음과 같은 절차를 따르면 됩니다:\n\n1. 신청서(별지 34호 서식)를 작성합니다.\n2. 여권 및 외국인등록증을 준비합니다.\n3. 수수료를 지불합니다.\n4. 고용허가서 사본을 제출합니다.\n5. 표준근로계약서 사본을 제출합니다.\n6. 사업자등록증 사본을 제출합니다.\n7. 입국 후 3년 만료 재고용에 따른 최대 1년 10개월 추가 연장의 경우 '취업기간만료자 취업활동기간 연장확인서(고용노동부 발급)'를 제출합니다.\n8. 체류지 입증서류(임대차계약서, 숙소제공 확인서, 체류기간 만료예고 통지우편물, 공공요금 납부영수증, 기숙사비 영수증 등)를 제출합니다.\n\n이러한 절차를 따르면 체류 기간을 연장할 수 있습니다. 그러나 이는 비전문취업자(E-9)에 대한 정보이며, 다른 체류자격에 따라 요구되는 서류와 절차가 다를 수 있습니다. 따라서 자신의 체류자격에 맞는 정보를 확인하는 것이 중요합니다.",
 'source_documents': [Document(page_content='체류자격\n변경허가\n●\n기타\n(\nG-1)\n자격\n소지자의\n비전문취업\n(\nE-9)\n자격\n회복절차\n○\n대상자\n■\n고용허가제\n등으로\n입국\n후\n산업재해\n치료\n등을\n위해\n기타\n(\nG-1)\n자격으로\n변경\n후\n국내체류\n중이던\n외국인근로자\n중\n국내\n체류기간\n상한이\n도래하지\n않은\n자\n○\n체류기간\n상한\n(\n입국일로부터\n기산\n)\n■\n비전문취업자\n(\nE-9)\n3\n년\n,\n다만\n3\n년\n만기\n체류자\n중\n고용주가\n재고용한\n경우\n1\n년\n10\n개월\n이내\n체류기간\n체류기간\n연장\n허가\n●\n제출\n서류\n○\n신청서\n(\n별지\n34\n호\n서식\n),\n여권\n및\n외국인등록증\n,\n수수료\n○\n고용허가서\n사본\n○\n표준근로계약서\n사본\n○\n사업자등록